In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer

# Load WikiText-103 dataset
# dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")

dataset = load_from_disk("../processed_datadir/wikitext-103-raw-train")

In [6]:
print(dataset.features)

{'text': Value(dtype='string', id=None)}


In [2]:
print(len(dataset))

1801350


In [2]:
print(dataset[1])

{'text': ' = Valkyria Chronicles III = \n'}


In [2]:
from datasets import Dataset, DatasetDict, ClassLabel, Features
story_data = {
    "text":[]
}
index = 1
while True:
    if dataset[index]["text"].count("=") == 2:
        story_data["text"].append(dataset[index]["text"])
    else:
        story_data["text"][-1] = story_data["text"][-1] + dataset[index]["text"]
    index += 1
    if index == len(dataset):
        break
    if index % 10000 == 1:
        print(index)
        


10001
20001
30001
40001
50001
60001
70001
80001
90001
100001
110001
120001
130001
140001
150001
160001
170001
180001
190001
200001
210001
220001
230001
240001
250001
260001
270001
280001
290001
300001
310001
320001
330001
340001
350001
360001
370001
380001
390001
400001
410001
420001
430001
440001
450001
460001
470001
480001
490001
500001
510001
520001
530001
540001
550001
560001
570001
580001
590001
600001
610001
620001
630001
640001
650001
660001
670001
680001
690001
700001
710001
720001
730001
740001
750001
760001
770001
780001
790001
800001
810001
820001
830001
840001
850001
860001
870001
880001
890001
900001
910001
920001
930001
940001
950001
960001
970001
980001
990001
1000001
1010001
1020001
1030001
1040001
1050001
1060001
1070001
1080001
1090001
1100001
1110001
1120001
1130001
1140001
1150001
1160001
1170001
1180001
1190001
1200001
1210001
1220001
1230001
1240001
1250001
1260001
1270001
1280001
1290001
1300001
1310001
1320001
1330001
1340001
1350001
1360001
1370001
1380001
1390

In [ ]:
for i in range(10):
    print(story_data["text"][i])

In [8]:
print(len(story_data["text"]))

29525


In [3]:


# 创建数据集
dataset = Dataset.from_dict(story_data)

# 保存数据集
# dataset.save_to_disk("../processed_datadir/wikitext-103-story-train")

In [ ]:
# read next fast speed
# dataset.save_to_disk("wikitext-103-raw-train")

In [ ]:
# for i in range(0, 1000):
#     print(dataset[i])

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../tokenizer_save/tokenizer-bert-base-uncased-512/")

In [6]:
import multiprocessing

num_proc = multiprocessing.cpu_count() - 4
print(num_proc)
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True,   # 在这就不带文本了
        max_length=tokenizer.model_max_length, return_overflowing_tokens=True,return_length=True)
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    
    for length, input_ids in zip(tokenized_inputs["length"], tokenized_inputs["input_ids"]):
        if length == tokenizer.model_max_length:
            input_ids_list.append(input_ids)
    for length, token_type_ids in zip(tokenized_inputs["length"], tokenized_inputs["token_type_ids"]):
        if length == tokenizer.model_max_length:
            token_type_ids_list.append(token_type_ids)
    for length, attention_mask in zip(tokenized_inputs["length"], tokenized_inputs["attention_mask"]):
        if length == tokenizer.model_max_length:
            attention_mask_list.append(attention_mask)
        
    return {"input_ids": input_ids_list, "token_type_ids": token_type_ids_list, "attention_mask": attention_mask_list}

# preprocess dataset
tokenized_datasets = dataset.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features

The max length for the tokenizer is: 512


Map (num_proc=16):   0%|          | 0/29525 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [11]:
# Split preprocessed dataset into train, validation, and test sets
splits = tokenized_datasets.train_test_split(test_size=0.05)
preprocessed_splits = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
    "test": load_from_disk("../processed_datadir/wikitext-103-story-test/")
})